# Warm Start Testing

In [1]:
import numpy as np
import gurobipy as gp
import math
import sympy
import contextlib
import time
import os
from utils import result, METHODS, INF, EPS
from mps_reader_preprocessor import read_mps_preprocess
from polyhedral_model import PolyhedralModel
from polyhedron import Polyhedron

## Read in Polyhedron P

In [2]:
# mps_fn='C:\Users\DillW\OneDrive - The University of Colorado Denver\Documents\GitHub\simplex_like_alg\netlib_lp_subset\kb2'
problem_dir = 'netlib_lp_subset'
problem = 'kb2'
mps_fn=os.path.join(problem_dir, problem)
results_dir='results'
max_time=300
sd_method='dual_simplex'
reset=False,
partition_polytope=False
n=0
k=0
spindle=False
spindle_dim=0
n_cone_facets=0
n_parallel_facets=0

In [3]:
### Build Initial Polyhedron from file
print(f'Reading {mps_fn}...')
c, B, d, A, b = read_mps_preprocess(mps_fn)
print('Building polyhedron...')
P = Polyhedron(B, d, A, b, c)

Reading netlib_lp_subset\kb2...
Building polyhedron...
Problem size: n = 41,  m_B = 77,  m_A = 16


## Get Initial Feasible Solution

In [4]:
print('Finding feasible solution...')
x_feasible = P.find_feasible_solution(verbose=False)

Finding feasible solution...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-01


## Perform SDCA

In [5]:
x = x_feasible
c=None
verbose=False
method='dual_simplex'
reset=False
max_time=300
first_warm_start=None
save_first_steps=0
problem_name=''

In [11]:
if c is not None:
    P.set_objective(c)
 
t0 = time.time()
x_current = x
if save_first_steps:
    np.save('solutions/{}_0.npy'.format(problem_name), x_current)      
active_inds = P.get_active_constraints(x_current)
    
pm = P.build_polyhedral_model(active_inds=active_inds, method=method)
####get edge for initial circuit direction here#########
x_feasible_2= P.second_vert(x, verbose=False, record_objs=True)
init_edge = np.array(x) - np.array(x_feasible_2)
########################################################

if first_warm_start is not None:
    print('Using custom warm start')
    pm.set_solution(first_warm_start)
t1 = time.time()
build_time = t1 - t0
print('Polyhedral model build time: {}'.format(build_time))
    
sub_times = {'sd': [], 'step': [], 'solve': [], 'phase_times': []}    
descent_circuits = []
obj_values = []
step_sizes = []
iter_times = []
simplex_iters = []
iteration = 0
obj_value = P.c.dot(x_current)
obj_values.append(obj_value)
t2 = time.time()
iter_times.append(t2 - t1)


# compute steepest-descent direction
descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(verbose=verbose)
simplex_iters.append(num_steps)
sub_times['solve'].append(solve_time)
sub_times['phase_times'].append(phase_times)
    
t3 = time.time()
sub_times['sd'].append(t3 - t2)
    
while abs(steepness) > EPS:
        
    t3 = time.time()
    if reset:
        pm.reset()
        
    # take maximal step
    x_current, alpha, active_inds = P.take_maximal_step(descent_direction, y_pos, y_neg)  
        
    if iteration % 50 == 0 or iteration == 1:
        print('\nIteration {}'.format(iteration))
        print('Objective: {}'.format(obj_value))
        print('Steepness: {}'.format(steepness))
        print('Step length: {}'.format(alpha))
        
    t4 = time.time()
    obj_value = P.c.dot(x_current)
    obj_values.append(obj_value)
    iter_times.append(t4 - t1)
    sub_times['step'].append(t4 - t3) 
    descent_circuits.append(descent_direction)
    step_sizes.append(alpha)     
                
    if math.isinf(alpha):
        # problem is unbounded
        result(status=1, circuits=descent_circuits, steps=step_sizes)
        
    # compute steepest-descent direction
    pm.set_active_inds(active_inds)
    descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(
                                                                                                verbose=verbose)
        
    t5 = time.time()
    sub_times['sd'].append(t5 - t4)
    sub_times['solve'].append(solve_time)
    sub_times['phase_times'].append(phase_times)
    simplex_iters.append(num_steps)
        
    iteration += 1
    current_time = t5 - t1
    if current_time > max_time:
        result(status=2)
    if iteration <= save_first_steps:
        np.save('solutions/{}_{}.npy'.format(problem_name, iteration), x_current)

t6 = time.time()
total_time = t6 - t1   
print('Total time for steepest-descent scheme: {}'.format(total_time))

Building polyhedral model. Solve method: dual_simplex
Set parameter Method to value 1
Polyhedral model built!
[-6.79665845e+01 -2.73719411e+00  0.00000000e+00 -3.35075261e+01
  0.00000000e+00 -3.03268274e+00 -5.61166531e+00  0.00000000e+00
  0.00000000e+00 -2.73719411e+00 -8.32876733e+00  0.00000000e+00
 -1.97103095e+01 -1.79221347e+03 -1.97789425e+03  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00307229e+01  0.00000000e+00
 -2.66712327e+01  0.00000000e+00 -3.67019556e+01 -3.13817226e+03
 -3.46200998e+03 -1.15543194e+01 -9.97528589e+02 -1.06511939e+03
 -6.96731726e+00 -2.61772504e+00  0.00000000e+00 -1.96927705e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.00000000e+01
 -8.22939035e+00  0.00000000e+00 -1.20000000e+01 -3.50000000e+01
 -6.79665845e+01]
Using custom warm start
(41,)
41
Set parameter Method to value 0
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: AMD Ryzen 7 7730U with Radeon Graphics, instruction set [SSE2|

## Getting Initial Feasible Direction

In [7]:
x_feasible_2

[59.99419042276329,
 1.9484432412522703,
 0.0,
 29.5771358784223,
 0.0,
 3.500509753312806,
 0.3396766290896038,
 0.0,
 4.561669527653692,
 1.9484432412522703,
 7.048016071292983,
 0.0,
 17.398315222601354,
 1599.4084069020323,
 1714.8749163307657,
 0.0,
 0.0,
 0.0,
 4.444878899585171,
 0.0,
 27.951983928707005,
 0.0,
 32.39686282829218,
 2740.17421779916,
 3134.0318457079343,
 10.199012371869758,
 884.4677360014057,
 927.8996181579719,
 6.499490246687185,
 0.7060705524214355,
 0.0,
 2.993451572761138,
 0.0,
 0.0,
 0.0,
 10.0,
 1.0457471815110393,
 0.0,
 12.0,
 35.0,
 59.99419042276329]